In [2]:
from data import MNQ, MES, CL, GC, ZC, ZB

In [3]:
import warnings

warnings.filterwarnings("ignore")

In [4]:
sessions = {
    "Tokyo Session": ("19:00", "04:00"),
    "London Session": ("03:00", "12:00"),
    "New York Session": ("7:00", "16:00")
}

In [11]:
import pandas as pd
import plotly.graph_objs as go
from datetime import datetime, timedelta

end_date = (MNQ.index.max() - timedelta(days=1)).date()
start_date = end_date - timedelta(days=4)
mnq_last_5_days = MNQ[(MNQ.index.date >= start_date) & (MNQ.index.date <= end_date)]

sessions = {
    "Tokyo Session": ("19:00", "04:00"),
    "London Session": ("03:00", "12:00"),
    "New York Session": ("07:00", "16:00")
}
session_colors = {
    "Tokyo Session": 'rgba(255, 182, 193, 0.3)', 
    "London Session": 'rgba(255, 255, 153, 0.3)', 
    "New York Session": 'rgba(173, 216, 230, 0.3)'
}

fig = go.Figure(data=[go.Candlestick(
    x=mnq_last_5_days.index,
    open=mnq_last_5_days['Open'],
    high=mnq_last_5_days['High'],
    low=mnq_last_5_days['Low'],
    close=mnq_last_5_days['Close'],
    name='MNQ'
)])

data_start = mnq_last_5_days.index.min().tz_localize(None)
data_end = mnq_last_5_days.index.max().tz_localize(None)

for session_name, (start, end) in sessions.items():
    session_color = session_colors[session_name]
    current_date = start_date
    
    while current_date <= end_date:
        start_time = datetime.combine(current_date, datetime.strptime(start, "%H:%M").time())
        end_time = datetime.combine(current_date + timedelta(days=1 if end < start else 0), datetime.strptime(end, "%H:%M").time())
        
        if end_time >= data_start and start_time <= data_end:
            fig.add_shape(
                type="rect",
                x0=max(start_time, data_start),
                x1=min(end_time, data_end),
                y0=mnq_last_5_days['Low'].min(),
                y1=mnq_last_5_days['High'].max(),
                fillcolor=session_color,
                opacity=0.3,
                line_width=0,
                layer="below"
            )
        
        current_date += timedelta(days=1)

fig.update_layout(
    title="MNQ Candlestick Chart with Trading Sessions (Last 5 Days from Yesterday)",
    xaxis_title="Date",
    yaxis_title="MNQ Price",
    xaxis_rangeslider_visible=False,
    template="plotly_dark"
)

fig.show()


In [36]:
import pandas as pd
import plotly.graph_objs as go
from datetime import datetime, timedelta

end_date = (MNQ.index.max() - timedelta(days=1)).date()
start_date = end_date - timedelta(days=4)
mnq_last_5_days = MNQ[(MNQ.index.date >= start_date) & (MNQ.index.date <= end_date)]

session_start = "13:00"
session_end = "16:00"

fig = go.Figure(data=[go.Candlestick(
    x=mnq_last_5_days.index,
    open=mnq_last_5_days['Open'],
    high=mnq_last_5_days['High'],
    low=mnq_last_5_days['Low'],
    close=mnq_last_5_days['Close'],
    name='MNQ'
)])

current_date = start_date
while current_date <= end_date:
    start_time = datetime.combine(current_date, datetime.strptime(session_start, "%H:%M").time())
    end_time = datetime.combine(current_date, datetime.strptime(session_end, "%H:%M").time())
    
    session_data = mnq_last_5_days.between_time(start_time.time(), end_time.time())
    session_data = session_data[session_data.index.date == current_date]
    
    if not session_data.empty:
        session_high = session_data['High'].max()
        session_low = session_data['Low'].min()
        session_range = session_high - session_low
        session_height = session_range

        fig.add_shape(
            type="rect",
            x0=start_time,
            x1=end_time,
            y0=session_low,
            y1=session_high,
            fillcolor="rgba(135, 206, 250, 0.2)",
            line=dict(color="orange", width=1)
        )
        
        fig.add_shape(
            type="line",
            x0=start_time,
            y0=session_high,
            x1=end_time,
            y1=session_high,
            line=dict(color="orange", width=2)
        )
        
        fig.add_shape(
            type="line",
            x0=start_time,
            y0=session_low,
            x1=end_time,
            y1=session_low,
            line=dict(color="orange", width=2)
        )
        
        for multiplier in [1, 2, -1, -2]:
            line_y = session_high + multiplier * session_height
            fig.add_shape(
                type="line",
                x0=end_time,
                y0=line_y,
                x1=end_time + (end_time - start_time),
                y1=line_y,
                line=dict(color="orange", width=1)
            )
        
        fig.add_annotation(
            x=start_time + (end_time - start_time) / 2,
            y=session_high + (session_high - session_low) * 1,
            text=f"Range: {session_range:.1f}<br>High: {session_high:.1f}<br>Low: {session_low:.1f}",
            showarrow=False,
            font=dict(color="white", size=10),
            align="center",
            bgcolor="rgba(0, 0, 0, 0)",
            bordercolor="rgba(0, 0, 0, 0)"
        )
    
    current_date += timedelta(days=1)

fig.update_layout(
    title="MNQ BLCK SESSION 1-4PM",
    xaxis_title="Date",
    yaxis_title="MNQ Price",
    xaxis_rangeslider_visible=False,
    template="plotly_dark"
)

fig.show()
